<a href="https://colab.research.google.com/github/tam8738/trai_cay/blob/main/train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np

In [ ]:
# Đường dẫn đến thư mục dữ liệu
root_dir = "/content/drive/MyDrive/trai_cay/data"

# Danh sách các thư mục con trong thư mục Training
train_dir = os.path.join(root_dir, "Training")
fruit_folders = sorted(os.listdir(train_dir))

In [ ]:
# Mảng chứa nhãn và tên file
categories = []
filenames = []

# Duyệt qua tất cả thư mục trong thư mục Training
for folder in fruit_folders:
    folder_path = os.path.join(train_dir, folder)

    # Kiểm tra nếu là thư mục
    if os.path.isdir(folder_path):
        # Xác định nhãn dựa trên tên thư mục
        label = folder  # Gán nhãn là tên thư mục

        # Đọc các file hình ảnh từ thư mục và các thư mục con
        for subdir, _, files in os.walk(folder_path):
            for file in files:
                if file.endswith(".jpeg") or file.endswith(".png") or file.endswith(".jpg"):
                    file_path = os.path.join(subdir, file)
                    filenames.append(file_path)
                    categories.append(label)

In [ ]:
# Tạo DataFrame từ danh sách tên file và nhãn
df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

# Chuyển đổi cột 'category' thành chuỗi
df["category"] = df["category"].astype(str)

# Kiểm tra các giá trị duy nhất trong cột 'category'
unique_categories = df['category'].unique()
print("Unique categories:", unique_categories)
print("Number of unique categories:", len(unique_categories))
print("Category counts:", df['category'].value_counts())

Unique categories: ['Apple' 'Banana' 'Grape' 'Mango' 'Strawberry']
Number of unique categories: 5
Category counts: category
Apple         1940
Banana        1940
Grape         1940
Mango         1940
Strawberry    1940
Name: count, dtype: int64


In [ ]:
# Chuyển đổi nhãn sang số nguyên và sau đó chuyển đổi lại thành chuỗi
df['category'] = df['category'].astype('category').cat.codes.astype(str)

# Kiểm tra kiểu dữ liệu của các cột
print(df.dtypes)

filename    object
category    object
dtype: object


In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)

# Đặt lại chỉ số của DataFrame
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

# Thiết lập các biến hỗ trợ cho việc huấn luyện
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size = 15

# In ra thông tin về dữ liệu để kiểm tra
print(f"Total training samples: {total_train}")
print(f"Total validation samples: {total_validate}")

Total training samples: 7760
Total validation samples: 1940


In [ ]:
# Định nghĩa các generator với kích thước ảnh nhỏ hơn và batch size nhỏ hơn
Image_W, Image_H = 100, 100  # Thay đổi kích thước ảnh nhỏ hơn để giảm thời gian xử lý

train_datagen = ImageDataGenerator(rotation_range=15,
                                   rescale=1./255,
                                   shear_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1)
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    x_col='filename',
                                                    y_col='category',
                                                    target_size=(Image_W, Image_H),
                                                    color_mode='rgb',  # Thêm 3 kênh màu (RGB)
                                                    class_mode='sparse',  # Sử dụng 'sparse' với sparse_categorical_crossentropy
                                                    batch_size=batch_size,
                                                    shuffle=True)  # Đảm bảo dữ liệu được xáo trộn

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              x_col='filename',
                                                              y_col='category',
                                                              target_size=(Image_W, Image_H),
                                                              color_mode='rgb',  # Thêm 3 kênh màu (RGB)
                                                              class_mode='sparse',  # Sử dụng 'sparse' với sparse_categorical_crossentropy
                                                              batch_size=batch_size,
                                                              shuffle=False)  # Không cần xáo trộn dữ liệu kiểm tra

Found 7760 validated image filenames belonging to 5 classes.
Found 1940 validated image filenames belonging to 5 classes.


In [ ]:
# Kích thước ảnh và số kênh màu
Image_W, Image_H, Image_C = 100, 100, 3  # 3 kênh màu (RGB)

# Xây dựng mô hình
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(Image_W, Image_H, Image_C)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(len(unique_categories), activation='softmax')  # Số lớp đầu ra phù hợp với số lượng class
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 98, 98, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 98, 98, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 49, 49, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 49, 49, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 47, 47, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 47, 47, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 23, 23, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 23, 23, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 21, 21, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 21, 21, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 10, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 10, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 12800)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │       6,554,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,652,869 (25.38 MB)

 Trainable params: 6,651,397 (25.37 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [ ]:
# Thiết lập các callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
]

In [ ]:
epochs = 10
history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=total_validate // batch_size,
                    steps_per_epoch=total_train // batch_size,
                    callbacks=callbacks)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.3712 - loss: 2.0466
Epoch 1: val_loss improved from inf to 1.32384, saving model to best_model.keras
517/517 ━━━━━━━━━━━━━━━━━━━━ 2751s 5s/step - accuracy: 0.3713 - loss: 2.0460 - val_accuracy: 0.4899 - val_loss: 1.3238
Epoch 2/10
  1/517 ━━━━━━━━━━━━━━━━━━━━ 4:07 480ms/step - accuracy: 0.4000 - loss: 1.6362

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_loss improved from 1.32384 to 1.30509, saving model to best_model.keras
517/517 ━━━━━━━━━━━━━━━━━━━━ 43s 83ms/step - accuracy: 0.4000 - loss: 1.6362 - val_accuracy: 0.4899 - val_loss: 1.3051
Epoch 3/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 516ms/step - accuracy: 0.4876 - loss: 1.3112
Epoch 3: val_loss improved from 1.30509 to 1.00891, saving model to best_model.keras
517/517 ━━━━━━━━━━━━━━━━━━━━ 336s 551ms/step - accuracy: 0.4877 - loss: 1.3111 - val_accuracy: 0.5886 - val_loss: 1.0089
Epoch 4/10
  1/517 ━━━━━━━━━━━━━━━━━━━━ 3:11 372ms/step - accuracy: 0.8000 - loss: 0.6050
Epoch 4: val_loss did not improve from 1.00891
517/517 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.8000 - loss: 0.6050 - val_accuracy: 0.5762 - val_loss: 1.0408
Epoch 5/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 514ms/step - accuracy: 0.5538 - loss: 1.1293
Epoch 5: val_loss did not improve from 1.00891
517/517 ━━━━━━━━━━━━━━━━━━━━ 283s 548ms/step - accuracy: 0.5538 - loss: 1.1292 - val_accuracy: 0.5592 - val_lo

In [ ]:
model.save('/content/drive/MyDrive/trai_cay/model.h5')


In [ ]:
# Import hàm load_model từ tensorflow.keras.models
from tensorflow.keras.models import load_model

# Tải mô hình đã lưu từ Google Drive
model_path = '/content/drive/MyDrive/trai_cay/model.h5'
model = load_model(model_path)

# Đánh giá mô hình bằng cách sử dụng tập dữ liệu kiểm tra
evaluation = model.evaluate(validation_generator, steps=total_validate // batch_size)
print(f"Validation Loss: {evaluation[0]}")
print(f"Validation Accuracy: {evaluation[1]}")

129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 141ms/step - accuracy: 0.6070 - loss: 0.9570
Validation Loss: 0.9454872012138367
Validation Accuracy: 0.6217054128646851
